In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [2]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [3]:
import random

# What are the types of dependencies in the following code?

### Create list size of 500 with integer between 0 and 9.

In [4]:
list = [random.randrange(10) for x in range(500)]

In [5]:
listrdd = sc.parallelize(list, 5)

In [6]:
pairs = listrdd.map(lambda x : (x,x*x))

In [7]:
pairs.collect()

[(6, 36),
 (9, 81),
 (8, 64),
 (6, 36),
 (3, 9),
 (8, 64),
 (7, 49),
 (7, 49),
 (2, 4),
 (3, 9),
 (4, 16),
 (1, 1),
 (9, 81),
 (9, 81),
 (0, 0),
 (4, 16),
 (1, 1),
 (3, 9),
 (6, 36),
 (1, 1),
 (7, 49),
 (4, 16),
 (5, 25),
 (0, 0),
 (1, 1),
 (4, 16),
 (0, 0),
 (1, 1),
 (3, 9),
 (5, 25),
 (1, 1),
 (5, 25),
 (4, 16),
 (3, 9),
 (7, 49),
 (5, 25),
 (2, 4),
 (5, 25),
 (9, 81),
 (5, 25),
 (0, 0),
 (9, 81),
 (7, 49),
 (6, 36),
 (3, 9),
 (9, 81),
 (2, 4),
 (5, 25),
 (8, 64),
 (2, 4),
 (8, 64),
 (5, 25),
 (3, 9),
 (4, 16),
 (8, 64),
 (4, 16),
 (8, 64),
 (8, 64),
 (8, 64),
 (5, 25),
 (2, 4),
 (9, 81),
 (8, 64),
 (8, 64),
 (7, 49),
 (8, 64),
 (6, 36),
 (4, 16),
 (5, 25),
 (8, 64),
 (5, 25),
 (5, 25),
 (0, 0),
 (0, 0),
 (8, 64),
 (0, 0),
 (0, 0),
 (5, 25),
 (0, 0),
 (9, 81),
 (4, 16),
 (8, 64),
 (0, 0),
 (0, 0),
 (3, 9),
 (8, 64),
 (3, 9),
 (9, 81),
 (2, 4),
 (7, 49),
 (4, 16),
 (8, 64),
 (2, 4),
 (1, 1),
 (2, 4),
 (6, 36),
 (1, 1),
 (6, 36),
 (9, 81),
 (6, 36),
 (3, 9),
 (0, 0),
 (2, 4),
 (8, 64),

In [8]:
reduced = pairs.reduceByKey(lambda x1,x2  : x1+x2)

### mapPartitions() : Return a new RDD by applying a function to each partition of this RDD.

In [9]:
finalRDD = reduced.mapPartitions(lambda itr : ["Key = " + str(key) +", Value = " + str(val) for (key,val) in itr])

In [10]:
finalRDD.collect()

['Key = 0, Value = 0',
 'Key = 5, Value = 1300',
 'Key = 6, Value = 1548',
 'Key = 1, Value = 43',
 'Key = 7, Value = 2940',
 'Key = 2, Value = 160',
 'Key = 8, Value = 3584',
 'Key = 3, Value = 495',
 'Key = 9, Value = 4698',
 'Key = 4, Value = 784']

## .toDebugString()

In [11]:
print(finalRDD.toDebugString().decode('utf-8'))

(5) PythonRDD[6] at collect at <ipython-input-10-35e5ca16387f>:1 []
 |  MapPartitionsRDD[5] at mapPartitions at PythonRDD.scala:133 []
 |  ShuffledRDD[4] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[3] at reduceByKey at <ipython-input-8-0a0b7f280310>:1 []
    |  PythonRDD[2] at reduceByKey at <ipython-input-8-0a0b7f280310>:1 []
    |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262 []


## reduceByKey() and sortByKey()

In [12]:
ordered_finalRDD = pairs.reduceByKey(lambda x1,x2  : x1+x2).sortByKey()

In [13]:
ordered_finalRDD.collect()

[(0, 0),
 (1, 43),
 (2, 160),
 (3, 495),
 (4, 784),
 (5, 1300),
 (6, 1548),
 (7, 2940),
 (8, 3584),
 (9, 4698)]

In [14]:
print(ordered_finalRDD.toDebugString().decode('utf-8'))

(5) PythonRDD[17] at collect at <ipython-input-13-66b0575ea070>:1 []
 |  MapPartitionsRDD[16] at mapPartitions at PythonRDD.scala:133 []
 |  ShuffledRDD[15] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[14] at sortByKey at <ipython-input-12-1c5113e4259d>:1 []
    |  PythonRDD[13] at sortByKey at <ipython-input-12-1c5113e4259d>:1 []
    |  MapPartitionsRDD[10] at mapPartitions at PythonRDD.scala:133 []
    |  ShuffledRDD[9] at partitionBy at NativeMethodAccessorImpl.java:0 []
    +-(5) PairwiseRDD[8] at reduceByKey at <ipython-input-12-1c5113e4259d>:1 []
       |  PythonRDD[7] at reduceByKey at <ipython-input-12-1c5113e4259d>:1 []
       |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262 []


## .coalesce() vs .repartition()

In [15]:
coalesce = ordered_finalRDD.coalesce(3)

In [16]:
print(coalesce.toDebugString().decode('utf-8'))

(3) CoalescedRDD[18] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  PythonRDD[17] at collect at <ipython-input-13-66b0575ea070>:1 []
 |  MapPartitionsRDD[16] at mapPartitions at PythonRDD.scala:133 []
 |  ShuffledRDD[15] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[14] at sortByKey at <ipython-input-12-1c5113e4259d>:1 []
    |  PythonRDD[13] at sortByKey at <ipython-input-12-1c5113e4259d>:1 []
    |  MapPartitionsRDD[10] at mapPartitions at PythonRDD.scala:133 []
    |  ShuffledRDD[9] at partitionBy at NativeMethodAccessorImpl.java:0 []
    +-(5) PairwiseRDD[8] at reduceByKey at <ipython-input-12-1c5113e4259d>:1 []
       |  PythonRDD[7] at reduceByKey at <ipython-input-12-1c5113e4259d>:1 []
       |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262 []


In [17]:
repartition = ordered_finalRDD.repartition(3)

In [18]:
print(repartition.toDebugString().decode('utf-8'))

(3) MapPartitionsRDD[23] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  CoalescedRDD[22] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  ShuffledRDD[21] at coalesce at NativeMethodAccessorImpl.java:0 []
 +-(5) MapPartitionsRDD[20] at coalesce at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[19] at RDD at PythonRDD.scala:53 []
    |  MapPartitionsRDD[16] at mapPartitions at PythonRDD.scala:133 []
    |  ShuffledRDD[15] at partitionBy at NativeMethodAccessorImpl.java:0 []
    +-(5) PairwiseRDD[14] at sortByKey at <ipython-input-12-1c5113e4259d>:1 []
       |  PythonRDD[13] at sortByKey at <ipython-input-12-1c5113e4259d>:1 []
       |  MapPartitionsRDD[10] at mapPartitions at PythonRDD.scala:133 []
       |  ShuffledRDD[9] at partitionBy at NativeMethodAccessorImpl.java:0 []
       +-(5) PairwiseRDD[8] at reduceByKey at <ipython-input-12-1c5113e4259d>:1 []
          |  PythonRDD[7] at reduceByKey at <ipython-input-12-1c5113e4259d>:1 []
          |  ParallelCollectionRDD

### Checkpoint : Checkpoint finalRDD and see its debug string.

In [19]:
print(repartition.toDebugString().decode('utf-8'))

sc.setCheckpointDir("checkpoint")
repartition.checkpoint() # all references to its parent RDDs will be removed.
print(repartition.isCheckpointed()) # not checkpointed until action()

repartition.count()
print(repartition.isCheckpointed())
print(repartition.getCheckpointFile())

(3) MapPartitionsRDD[23] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  CoalescedRDD[22] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  ShuffledRDD[21] at coalesce at NativeMethodAccessorImpl.java:0 []
 +-(5) MapPartitionsRDD[20] at coalesce at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[19] at RDD at PythonRDD.scala:53 []
    |  MapPartitionsRDD[16] at mapPartitions at PythonRDD.scala:133 []
    |  ShuffledRDD[15] at partitionBy at NativeMethodAccessorImpl.java:0 []
    +-(5) PairwiseRDD[14] at sortByKey at <ipython-input-12-1c5113e4259d>:1 []
       |  PythonRDD[13] at sortByKey at <ipython-input-12-1c5113e4259d>:1 []
       |  MapPartitionsRDD[10] at mapPartitions at PythonRDD.scala:133 []
       |  ShuffledRDD[9] at partitionBy at NativeMethodAccessorImpl.java:0 []
       +-(5) PairwiseRDD[8] at reduceByKey at <ipython-input-12-1c5113e4259d>:1 []
          |  PythonRDD[7] at reduceByKey at <ipython-input-12-1c5113e4259d>:1 []
          |  ParallelCollectionRDD

In [20]:
print(repartition.toDebugString().decode('utf-8'))

(3) MapPartitionsRDD[23] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  ReliableCheckpointRDD[25] at count at <ipython-input-19-6e9f129a1a34>:7 []


In [21]:
sc.stop()